> **DO NOT EDIT IF INSIDE `computational_analysis_of_big_data_2018_spring` folder** 

# Week 5: Machine Learning 2

*Thursday, February 22, 2018*

There are innumerable machine learning models (and algorithms for fitting them to data) out there, and each one has something special about it that makes it suitable to a specific type of problem. To apply machine learning and get some initial results is fairly straight forward. Getting under the hood, however, requires a bit of work. This week we will look into how Decision Trees and PCA works. In the exercises today you will:

* Implement a standard decision tree mechanism
* Use PCA to visualize clusters in your data

**Questions**: Outside of class, use [issue on GitHub](https://github.com/ulfaslak/computational_analysis_of_big_data_2018_spring/issues) for asking questions.

**Feedback:** Am I going too fast over things? Overexplaining? Send me anonymous feedback about the exercises, lectures and course in general at http://ulfaslak.com/vent.

## Exercises

**Get started**: Load the data you used last week to classify hero/villainness. It should consist of two arrays, one of dimensions ($N_{characters}$, $N_{teams}$) which is your feature matrix that has the team alliances of each character as a row vector of ones and zeros, and another of dimensions ($N_{characters}$, ) which is your target array that gives whether a character is a hero (1) or a villain (0). You can either load the data or copy/paste the code that generates it.

*Hint: If you had trouble with the data, use mine. You can load it as a `pandas.DataFrame`, with `pd.read_csv('data_team_alliances.csv', index_col=0)`. The rightmost column is the target array. Do not include this in PCA (think about why)!.*

In [1]:
import re, os
# Get alliances function
def get_alliances(char, faction=None):
    """Return list of alliances for Marvel character."""
    
    if faction is None:
        for faction in ["heroes", "ambiguous", "villains"]:
            faction_chars = [c[:-4] for c in os.listdir("../data/%s" % faction)]
            if char in faction_chars:
                break
    
    # Load character markup
    with open("../data/%s/%s.txt" % (faction, char)) as fp:
        markup = fp.read()

    # Get alliance field
    alliances_field = re.findall(r"alliances[\w\W]+?\n", markup)
    if alliances_field == []:
        return []

    # Extract teams from alliance field
    return [t[2:-1] for t in re.findall(r"\[\[.+?[\]\|]", alliances_field[0][10:])]

In [2]:
# Get a set of teams
teams = set()
for faction in ["heroes", "ambiguous", "villains"]:

    faction_chars = [c[:-4] for c in os.listdir("../data/%s" % faction) if c != ".DS_Store"]
    for char in faction_chars:
        teams.update(get_alliances(char)) # set of all alliances for every character

In [3]:
import numpy as np
index_dict = dict(zip(list(teams), range(len(teams)))) # teams and their index

def get_alliance_vect(char):
    char_teams = get_alliances(char) # for a certain character, get his alliances
    alliance_vector = [0] * len(teams) # cteate list of zeroes for all teams
    # for each character team the character has
    # get that index
    # and set the alliance vector to 1
    for ct in char_teams:
        team_index = index_dict[ct]
        alliance_vector[team_index] = 1
    # return vector as np array
    return np.array(alliance_vector)

In [6]:
X = [] # 1 Characters per row; 1 alliance per column
y = []
for faction in ["heroes", "villains"]:
    for char in [c[:-4] for c in os.listdir("../data/%s" % faction) if c != ".DS_Store"]:
        if sum(get_alliance_vect(char)) == 0:
            # skip character has no alliances
            continue
        # Add character's alliances vector to X
        X.append(get_alliance_vect(char))
        if faction == "heroes":
            y.append(1)
        else:
            y.append(0)
# Character alliance matrix
X = np.array(X)
y = np.array(y)

### Part 1: Decision trees

In this exercise you will implement the decision making algorithm of a decision tree classifier, step by step.

>**Ex. 5.1.1**: Read about [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory).
1. What is it? How is it defined mathematically (write out the formula in LateX formatting)?
2. Write a function that computes the Shannon-entropy of a probability vector. Compute the Shannon entropy of `p=[0.4, 0.6]`.

Information entropy - average amount of information produced by a stochastic source of data.
Think "The measure of information entropy associated with each possible data value is the negative logarithm of the probability mass function for the value"

"The entropy provides an absolute limit on the shortest possible average length of a lossless compression encoding of the data produced by a source, and if the entropy of the source is less than the channel capacity of the communication channel, the data generated by the source can be reliably communicated to the receiver"

In [7]:
import math
def ShannonEntropy(v):
    # Shannon entropy is the sum of -p*log(p) for all events
    return sum([-1*i * math.log(i, 2) for i in v])

p = [0.4, 0.6]
print ShannonEntropy(p)

0.970950594455


>**Ex. 5.1.2**: Split your data into two subsets. One where characters are affiliated with X-men and one where they are not.
1. What is the entropy of target labels in each subset?
2. What is the weighted average entropy of the split?
3. Write a function that computes the weighted average entropy of a split, given the data and team (name or id) on which to split the data.
4. Plot the distribution of split entropy for all features. Comment on the result. My figure looks [like this](http://ulfaslak.com/computational_analysis_of_big_data/exer_figures/example_6.2.2.4.png).

In [18]:
# Get team index for Xmen
for i,team in enumerate(teams):
    if team == "X-Men": break
X_ind = i
# Count xmen and non xmen
X_men = list()
y_men = list()
Not_X_men = list()
Not_y_men = list()
# Loop through all chars
for i, char in enumerate(X):
    if char[X_ind]: 
        X_men.append(char)
        y_men.append(y[i])
    else: 
        Not_X_men.append(char) 
        Not_y_men.append(y[i])
# Calculate entropy of each target label
# p3 = [len(X_men)/float(len(X)), len(Not_X_men)/float(len(X))]
X_men_probs = sum(y_men)/float(len(y_men)), 1 - sum(y_men)/float(len(y_men))
Not_X_men_probs = sum(Not_y_men)/float(len(Not_y_men)), 1 - sum(Not_y_men)/float(len(Not_y_men))
X_e = ShannonEntropy(X_men_probs)
N_X_e = ShannonEntropy(Not_X_men_probs)
# Print
print "X-men subset entropy: " + str(X_e)
print "Non-X-men subset entropy:" + str(N_X_e)

X-men subset entropy: 0.34351974100740135
Non-X-men subset entropy:0.9936846391950632


In [21]:
# Weighted average entropy of the split
weighted_average_entropy = (X_e*len(X_men) + N_X_e*len(Not_X_men))/(len(X))
print "weight average entropy of the split: " + str(weighted_average_entropy)

weight average entropy of the split: 0.9514591087548987


In [79]:
def compute_average_entropy(X, y, teams, team_name):
    # Get team index
    for i,team in enumerate(teams):
        if team == team_name: break
    X_ind = i
    # Count xmen and non xmen
    X_in = list()
    y_in = list()
    X_out = list()
    y_out = list()
    # Loop through all chars
    for i, char in enumerate(X):
        if char[X_ind]: 
            X_in.append(char)
            y_in.append(y[i])
        else: 
            X_out.append(char) 
            y_out.append(y[i])
    # Calculate entropy of each target label
    if len(y_in) == 0 or len(y_out) == 0: return 1. # ********** Shady 
    X_in_probs = sum(y_in)/float(len(y_in)), 1 - sum(y_in)/float(len(y_in))
    X_out_probs = sum(y_out)/float(len(y_out)), 1 - sum(y_out)/float(len(y_out))
    if 0. in X_in_probs or 0. in X_out_probs: return 1. # ********** Shady
    X_in_e = ShannonEntropy(X_in_probs)
    X_out_e = ShannonEntropy(X_out_probs)
    # Weighted average entropy of the split
    weighted_average_entropy = (X_in_e*len(X_in) + X_out_e*len(X_out))/(len(X))
    return weighted_average_entropy

In [77]:
print compute_average_entropy(X, y, teams, "X-Men")

0.9514591087548987


In [78]:
team_e_splits = dict()
for team in teams:
    team_e_splits[team] = compute_average_entropy(X, y, teams, team)
print team_e_splits

{'A.R.M.O.R.': 1.0, "N'astirh": 1.0, 'Nanny (comics)': 1.0, 'Headmen': 1.0, 'Brotherhood of Evil Mutants': 0.9992281215957418, 'X-Factor Investigations': 1.0, 'Young X-Men': 1.0, 'New Invaders': 1.0, 'Hand (comics)': 0.999291099456926, 'Indian Police Service': 1.0, 'Triad (underground society)': 1.0, 'Hellions (comics)#The New Hellions': 0.99914605670267, 'Riot Squad (comics)': 1.0, 'Ultimates': 1.0, 'Genoshan Magistrates': 1.0, 'Selene (comics)': 1.0, 'Darkhold': 1.0, "Nick Fury's Howling Commandos": 0.9973439817327512, 'Super-Sentinels': 0.9993141095593, 'Crazy Gang (Marvel Comics)': 0.9993141095593, "Deep Six (Marvel Comics)#Namor's Deep Six": 1.0, 'New Warriors': 0.9711606991457097, 'List of Marvel Comics demons#The Hell-lords': 1.0, 'Vampire (Marvel Comics)': 1.0, 'Secret Empire (comics)': 1.0, 'Xavier Institute Student Body#Corsairs': 1.0, 'Dark Avengers': 0.9976288492095544, 'Mutant X (comics)': 1.0, 'Hydra (comics)': 0.986706243568707, 'Revengers': 0.99914605670267, 'Theatre of

>**Ex. 5.1.3**: Print the maximum entropy path of a decision tree.
>
>1. Implement the following pseudocode and print the output:<br><br>
>Step 1. Find `team` that gives lowest split entropy for `data`. Print `team`.<br>
>Step 2. Split `data` on `team`, to produce `data0` and `data1`. Print the entropy of each, as well as their weighted avg. entropy.<br>
>Step 3. Overwrite the `data` variable with either `data0` or `data1`, depending on which has the highest entropy.<br>
>Step 4. Stop if there are less than 5 datapoints in `data`. Otherwise start over from 1.<br><br>
>My output looks [like this](http://ulfaslak.com/computational_analysis_of_big_data/exer_figures/example_6.2.3.1.png) for the first five splits.<br><br>
>
>2. Comment on decision path your code takes: How splits are there? Do you notice anything interesting about the final splits? Why do we choose to stop splitting before `data` get smaller than 5?
>3. Train a `sklearn.tree.DecisionTreeClassifier` classifier on the dataset. Initiate the classifier with `criterion='entropy'`. What are the most important features of this classifier? How does this line up with the order of the order of splits you just printed (a comment is fine)?

In [86]:
# 1.) Lowest split entroy
minimum_split = 1.
min_team = ""
for key in team_e_splits:
    if team_e_splits[key] < minimum_split:
        minimum_split = team_e_splits[key]
        min_team = key
print "Lowest split: " + min_team
# 2.) 
print compute_average_entropy(X, y, teams, min_team)

Lowest split: Avengers (comics)
0.9505869637306638


### Principle component analysis (PCA)

The point of this exercise is to learn how to visualize high-dimensional data, and get a feeling of how your Marvel data looks when projected to a plane.

>**Ex. 5.2.1**: Apply a PCA to your data. If `X` is your feature matrix, a PCA can be done like:

>        pca = sklearn.decomposition.PCA()
>        pca.fit(X)
>        X_pca = pca.transform(X)

>1. What is the dimensionality of `X_pca` compared to `X`? What happened to `X` when you transformed it?
2. Plot the first two components/columns of the transformed data and color the points by their class label. My plot looks [like this](http://ulfaslak.com/computational_analysis_of_big_data/exer_figures/example_6.1.1.2.png). Comment on the result. What would plotting two other components against each other show you?
3. Plot the explained variance ratio of each component. What does this tell you about the dataset? My plot looks [like this](http://ulfaslak.com/computational_analysis_of_big_data/exer_figures/example_6.1.1.3.png).

>*Hint for 2: `plt.scatter` takes an argument `color` which must receive either a string such as `red` or `blue`, or a list of rgb values or strings such as `['red', 'blue', 'blue', ...]`.*

In [12]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(X)
X_pca = pca.transform(X)
print(X_pca)

[[-1.26523052e-01 -3.43691208e-02 -1.29205686e-01 ... -1.28030724e-16
   7.31294365e-17 -1.33299269e-16]
 [-1.24676798e-01 -6.09176717e-02 -1.04788984e-01 ...  7.85639999e-17
  -5.29632761e-17  2.84636952e-17]
 [-1.17666228e-01 -6.77514415e-02 -9.77505749e-02 ... -4.31919040e-17
  -4.85451523e-17 -1.85906791e-17]
 ...
 [-1.54304635e-01  2.15674970e-01  4.54697385e-01 ...  3.12927852e-17
  -1.50162001e-17  2.82468547e-17]
 [-1.45322338e-01 -5.16844087e-02 -3.25874370e-02 ...  1.07878116e-17
   3.47757847e-17  9.75544786e-17]
 [-1.61956482e-01 -4.88637108e-03  2.37140920e-02 ...  2.11419424e-18
   7.18554990e-17  1.77267055e-17]]
